## Part 2 - Language Translation

Installing the necessary packages for the Language identification and translation

In [0]:
!pip install ftfy
!pip install mxnet
!pip install googletrans

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for ftfy: filename=ftfy-5.7-cp36-none-any.whl size=44593 sha256=c2153a003ca2efa92d549292428913f05456c7b755bfc6ade16417915c2e0cca
  Stored in directory: /root/.cache/pip/wheels/8e/da/59/6c8925d571aacade638a0f515960c21c0887af1bfe31908fbf
Successfully built ftfy
     |████████████████████████████████| 68.7MB 54kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
  Created wheel for googletrans: filename=googletrans-2.4.0-cp36-none-any.whl size=15777 sha256=5f1f984ed6dfe405894ae2e2caa5bd581d8d9061709072e74de1248d90b63206
  Stored in directory: /root/.cache/pip/wheels/50/d6/e7/a8efd5f2427d5eb258070048718fa56ee5ac57fd6f53505f95
Successfully built googletrans


In [0]:
import pprint, pickle
import pandas as pd
from nltk.tokenize import RegexpTokenizer 
import unicodedata
from ftfy import fix_encoding, fix_text, badness
from googletrans import Translator

Mounting the google drive and setting the working directory

In [0]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Change the current working Directory 
import os

path = '/content/drive/My Drive/LanguageTranslation'
try:   
    os.chdir(path)
    print("Directory changed to Project folder")
except OSError:
    print("Can't change the Current Working Directory")  

Directory changed to Project folder


The code below will read and load the pickled file. This file has only those records that are not in English. This pickled file has been created from the main the main preprocessing file. All the non-english dataset has been separated out and pickled.

In [0]:
pkl_file = open('non_eng_data.pkl', 'rb')
non_eng_data = pickle.load(pkl_file)

pkl_file.close()

In [0]:
non_eng_data.head()

,Short description,Description,Caller,Assignment group
99,password expiry tomorrow,\n\nreceived from: ecprjbod.litmjwsy@gmail.com...,ecprjbod litmjwsy,GRP_0
116,server issues,\r\n\r\nreceived from: bgqpotek.cuxakvml@gmail...,bgqpotek cuxakvml,GRP_0
124,mobile device activation,"from: tvcdfqgp nrbcqwgj \nsent: friday, octobe...",tvcdfqgp nrbcqwgj,GRP_0
164,æ’¤å›ž: ticket_no1564867 -- comments added,\n\nreceived from: abcdri@company.com\n\nwindy...,tycludks cjofwigv,GRP_0
170,[urgent!!] delivery note creation request!!,\n\nreceived from: fbvpcytz.nokypgvx@gmail.com...,fbvpcytz nokypgvx,GRP_18


Creating a pivot table to analyse as to how many records belong to each group which are not in English. Need this information as we had sufficient Grp_0 records for training but other GRPs did not have sufficient data. We will do the language identification and identication only for Non Grp_0 records.

In [0]:
table = pd.pivot_table(non_eng_data,index="Assignment group",values='Caller',margins_name='Total',margins=True,aggfunc='count',fill_value=0)
print(table)

                  Caller
Assignment group        
GRP_0                261
GRP_1                  1
GRP_10                12
GRP_11                 4
GRP_12                34
GRP_13                13
GRP_14                 6
GRP_15                 2
GRP_16                 9
GRP_17                 3
GRP_18                15
GRP_19                 8
GRP_2                 19
GRP_20                 2
GRP_21                 3
GRP_22                 2
GRP_23                 4
GRP_24                87
GRP_25                20
GRP_26                 5
GRP_27                 1
GRP_28                10
GRP_29                13
GRP_3                  3
GRP_30                35
GRP_31                43
GRP_32                 3
GRP_33                52
GRP_34                17
GRP_37                 2
GRP_38                 1
GRP_39                 3
GRP_4                 16
GRP_40                 3
GRP_41                 1
GRP_42                23
GRP_44                 1
GRP_45                 3


Installing the Fasttext that has the model to do 176 language identification

In [0]:
pip install fasttext

     |████████████████████████████████| 61kB 1.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2386524 sha256=4a08d8edf1c2f2079c06328f9d4e26e35218afa8abec78563981b5e8ae2ff4f6
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


The below text loads the lid.176.bin model that identifies the lanaguage of the text

In [0]:
import fasttext
import fasttext.util
##fasttext.util.download_model('en', if_exists='ignore')  # English

ft = fasttext.load_model('lid.176.bin')

The predict_lang method will identify the language of the text that has been passed

In [0]:
def predict_lang(model,texts): return model.predict(texts,k=1)

In [0]:
tokenizer = RegexpTokenizer("[\w']+") 
def predictLanguage(texts):
  for word in tokenizer.tokenize(texts):
    prediction = predict_lang(ft,word)
    label = prediction[0][0].split("__label__")[1]
  return label

The below code snippet will loop through all the records and then
1. Identify the language of the text
2. Translate that to English
3. Create a new DataFrame with the translated text

All the above are done only for Non GRP_0 records as we have sufficient non GRP_0 records for training the model

In [0]:
translator = Translator()
i=0
trans_non_eng_data = pd.DataFrame(columns=['Short description', 'Description', 'Caller','Assignment group'])

for index, row in non_eng_data.iterrows(): 
  if(row['Assignment group']!='GRP_0'):
    shortDesc= fix_text(row['Short description'])
    desc=fix_text(row['Description'])
    newDesc=desc.replace("\n"," ")
    label = predictLanguage(shortDesc)
    if(label!='en'):
      text = shortDesc +'|'+ newDesc
      transText = translator.translate(text,'en')
      transText = transText.text
      transShortDesc, transDesc = transText.split("|",maxsplit=1)
      print(shortDesc, label)
      i=i+1
      trans_non_eng_data=trans_non_eng_data.append(pd.Series([transShortDesc,transDesc,row['Caller'],row['Assignment group']], index=trans_non_eng_data.columns ), ignore_index=True)
    else:
      trans_non_eng_data=trans_non_eng_data.append(pd.Series([row['Short description'],row['Description'],row['Caller'],row['Assignment group']], index=trans_non_eng_data.columns ), ignore_index=True)
    if(i==500):
      break


trans_non_eng_data.shape

unable to access password_management_tool id password manager ijeqpkrz nwtehsyx grauw nl
青岛兴合机电shipment notification邮箱设置 ja
support für fa.gstry \arexjftu ohxdwngl de
probleme mit bluescreen . ru
support für we111\zlqfptjx xnklbfua de
EU_tool aktualisierung alle 15 min läuft nicht    de
kalendereinträge - qdxyifhj zbwtunpy pl
support für fa.thrydsss-funke \laeusvjo fvaihgpx fi
hpqc delivers error message: "user is not maintained in the project fy_13 release. please contact hpqc admin" ca
telefon gigaset m2 ex professional (tel.:) läd nicht mehr. anschluss defekt. de
timerecording terminals in plant germany no communication to the server in fürth(bcom server in fürth) de
ie浏览器打开crm系统后提示用户已被注销,无法登录crm系统。 zh
 server lnbdm839 (active directory) located in   philadelph​ia is down since 12:25 am et on 10/25. de
zeitdaten für germany steel fehlen de
msd - office 2013 outlook 打不开,显示无法创建新的堆栈防护页面 zh
druckerfunktionsstörung de
pc der maschine r45 defekt de
EU_tool in germany steel ohne funktion -

(567, 4)

In [0]:
trans_non_eng_data.head(50)

,Short description,Description,Caller,Assignment group
0,[urgent!!] delivery note creation request!!,\n\nreceived from: fbvpcytz.nokypgvx@gmail.com...,fbvpcytz nokypgvx,GRP_18
1,unable to access password_management_tool id p...,i try to change now my password acc. to attach...,ijeqpkrz nwtehsyx,GRP_2
2,Qingdao-the Electrical and Mechanical-mail shi...,"from: sent: friday, october 28, 2016 7:20 am ...",yafxlpwi lhxvatkb,GRP_18
3,support for fa.gstry \ arexjftu ohxdwngl,support for fa.konnica \ arexjftu ohxdwngl,arexjftu ohxdwngl,GRP_24
4,problems with blue screen,". hello, it happened again. the pc hanged hims...",vrfpyjwi nzhvgqiw,GRP_24
5,erp pi and msd crm connectivity issue- serirtc...,hi all\n\nwe have a connectivity issue between...,kgytujhe bonhwzrx,GRP_14
6,printer problem / issue information,please complete all required questions below. ...,dzjxrkae grqczsmx,GRP_3
7,"a kÃ¼ndigung for fgxprnub hlanwgqj, 11161827, ...","a kÃ¼ndigung for fgxprnub hlanwgqj, 11161827, ...",ucawbivs ountxzir,GRP_2
8,outlookæ”¶åˆ°ç®±ä¸­folderå˜ä¸ºæ¯å¤©ä¸€ä¸ªfol...,outlookæ”¶åˆ°ç®±ä¸­folderå˜ä¸ºæ¯å¤©ä¸€ä¸ªfol...,bxfdkiol mdqlszvc,GRP_30
9,support for we111 \ zlqfptjx xnklbfua,support for we111 \ zlqfptjx xnklbfua,zlqfptjx xnklbfua,GRP_24


Re-pickle all the translated non English, non GRP_0 records. This pickle file to be merged with the Original English records. This final data will be used for furthur preprocessing and fed into the model for training the model

In [0]:
output = open('trans_non_eng_data.pkl', 'wb')

pickle.dump(trans_non_eng_data, output)

output.close()